In [2]:
import os
import pandas as pd
import h5py
import re

In [3]:
# Parse METAR report 
def parse_metar_line(date_time, line):
    """Parse a single METAR report line into its components, including optional AUTO."""
    metar_pattern = re.compile(
        r'^(?P<station>[A-Z]{4})\s+'
        r'(?P<datetime>\d{2}\d{4}Z)\s+'
        r'(?P<auto>AUTO\s+)?'
        r'(?P<wind>\d{3}\d{2}(G\d{2})?(KT|MPS|KMH))?\s*'
        r'(?P<visibility>\d{4}(SM|NDV)?|CAVOK|////)?\s*'
        r'(?P<clouds>(///\d{3}///|FEW|SCT|BKN|OVC|NSC|VV|///)\d{0,3}[A-Z]{0,3}\s*)*'
        r'(?P<temperature>M?\d{2})/(?P<dewpoint>M?\d{2}|//)\s+'
        r'Q(?P<pressure>\d{4})\s*'
        r'(?P<remarks>.+)?$'
    )
    
    match = metar_pattern.match(line)
    if match:
        data = match.groupdict()
        
        # Process the clouds group to handle multiple cloud layers
        clouds = data.get('clouds')
        if clouds:
            data['clouds'] = clouds.strip().split()
        else:
            data['clouds'] = None
        
        data['date_time'] = date_time
        # Handle optional fields
        data['auto'] = data['auto'].strip() if data['auto'] else None
        data['remarks'] = data['remarks'].strip() if data['remarks'] else None
        
        return data
    
    return None

# Parse TAF report 
def parse_taf_block(block):
    """Parse a full TAF block into its components."""
    # Regex pattern to match main TAF components across multiple lines
    taf_pattern = re.compile(
        r'(?P<station>TAF \w{4})\s+'                     # Station identifier (TAF + ICAO code)
        r'(?P<datetime>\d{6}Z)\s+'                       # Date and time of issuance
        r'(?P<validity>\d{4}/\d{4})\s+'                  # Validity period
        r'(?P<wind>(\d{5}(KT|MPS|KMH)|VRB\d{2}(KT|MPS|KMH)?)?)\s*'  # Wind information
        r'(?P<visibility>CAVOK|\d{4})?\s*'               # Visibility, including CAVOK
        r'(?P<clouds>(SCT|BKN|FEW|NSC|OVC|NSC|VV)\d{3}[A-Z]{0,3})?\s*' # Cloud information
        r'(?P<temp_dewpoint>(TX|TN)\d{2}/\d{4}Z)?\s*'    # Max/min temperature forecast
        
        # Additional forecasts as separate groups
        r'(?P<probability>(PROB\d{2}\s+\d{4}/\d{4}\s+.*)*)'     # Probability forecasts (e.g., PROB30)
        r'(?P<temporary>(TEMPO\s+\d{4}/\d{4}\s+.*)*)'        # Temporary forecasts
        r'(?P<becoming>(BECMG\s+\d{4}/\d{4}\s+.*)*)'        # Becoming forecasts
        r'(?P<from_forecasts>(FM\d{6}\s+.*)*)'                       # From forecasts
    )
    
    match = taf_pattern.search(block)
    if match:
        return match.groupdict()
    return None

# CWAM
def get_dataset(f):
    data_entries = []
    for forecast_time in f['Deviation Probability']:
        for flight_level in f[f'Deviation Probability/{forecast_time}']:
            for contour in f[f'Deviation Probability/{forecast_time}/{flight_level}']:
                for threshold in f[f'Deviation Probability/{forecast_time}/{flight_level}/{contour}']:
                    for polygon in f[f'Deviation Probability/{forecast_time}/{flight_level}/{contour}/{threshold}']:
                        # Construct the full path for the dataset
                        dataset_name = f'Deviation Probability/{forecast_time}/{flight_level}/{contour}/{threshold}/{polygon}'
                        
                        dataset = f[dataset_name][:]
                        latitudes, longitudes = dataset[0], dataset[1]
                        
                        fcst = forecast_time
                        flvl = flight_level
                        trsh = threshold
                        poly = polygon

                        data_entries.append({
                            "Forecast Time (FCST)" : fcst,
                            "Flight Level (FLVL)" : flvl,
                            "Threshold (TRSH)" : trsh,
                            "Polygon Number (POLY)" : poly,
                            "Latitudes" : latitudes,
                            "Longitudes" : longitudes
                        })
    return pd.DataFrame(data_entries)

In [4]:
def load_data(data_type, dataset_purpose, path_level=None, month=None, day=None, file_name=None, base_dir=None):
    """
    Loads a data file from a specified path.
    
    Parameters
    - data_type (str): Data type, e.g., "CWAM", "FUSER", "METAR", "TAF".
    - dataset_purpose (str): "train" or "test", indicating training or testing data.
    - path_level (str): CWAM or METAR part_X, e.g., "part_1".
    - month (str): Month, e.g., "09".
    - day (str): Day, e.g., "29".
    - file_name (str): Filename (without extension or type-specific suffix), e.g., "2022_09_29_20_00_GMT.Forecast".
    - base_dir (str): Root directory of the data, e.g., "/home/finalProject/data".
    
    Returns:
    - DataFrame, HDF5 file object, or string content based on file type.
    """

    base_dir = "/home/jaosn/finalProject/data" if not base_dir else base_dir
    # Specify file extensions
    file_ext = file_extension[data_type]
    
    # Build the file path based on data type
    if data_type == "CWAM":
        # CWAM Path: data/CWAM/test/part_X/MM/DD
        if not (path_level and month and day):
            raise ValueError("CWAM data requires path_level, month, and day")
        path = os.path.join(base_dir, data_type, dataset_purpose, path_level, month, day)
        # CWAM files use a specific naming convention
        file_path = os.path.join(path, f"{file_name}.h5.CWAM.h5")
        
    elif data_type == "FUSER":
        # FUSER Path: data/FUSER/train/KXXX/
        if not path_level:
            raise ValueError("FUSER data requires fuser_type")
        path = os.path.join(base_dir, data_type, dataset_purpose, path_level)
        file_path = os.path.join(path, f"{file_name}.csv")
    
    elif data_type in ["METAR", "TAF"]:
        # METAR and TAF Paths: data/METAR/train/part_X/
        if dataset_purpose == "train" and not path_level:
            raise ValueError(f"{data_type} data requires path_level")
        if dataset_purpose == "train":
            path = os.path.join(base_dir, data_type, dataset_purpose, path_level)
        else:
            path = os.path.join(base_dir, data_type, dataset_purpose)
        file_path = os.path.join(path, f"{file_name}.txt")
    
    else:
        raise ValueError(f"Unsupported data type: {data_type}")

    
    # Check if file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    # Load file based on file type
    if data_type == "FUSER":
        def extract_file_type(file_name):
            pattern = r'^(?P<airport>\w+)' \
                      r'_(?P<date_range>\d{4}-\d{2}-\d{2}(_\d{4}-\d{2}-\d{2})?)' \
                      r'\.(?P<file_type>\w+)_data_set$'
              
            match = re.match(pattern, file_name)
            
            if match:
                file_type = match.group('file_type')
                return file_type
            else:
                return None
        
        df = pd.read_csv(file_path)
        file_type = extract_file_type(file_name)
        df['file_type'] = file_type
        return df
    elif data_type in ["METAR", "TAF"]:
        if data_type == "METAR":
            with open(file_path, 'r') as file:
                lines = file.readlines()
                lines = [line.strip() for line in lines if line.strip()]
                data_entries = []
                date_time = None  # To keep track of the current date and time

                for line in lines:
                    # Check if the line is a date line
                    if re.match(r'\d{4}/\d{2}/\d{2} \d{2}:\d{2}', line):
                        date_time = line  # Store the date and time
                    elif date_time:  # If we have a date_time, process the METAR line
                        parsed_data = parse_metar_line(date_time, line)
                        if parsed_data:
                            data_entries.append(parsed_data)
                        date_time = None 
            return pd.DataFrame(data_entries)
        else:
            with open(file_path, 'r', encoding='ISO-8859-1') as file:
                lines = file.readlines()
    
            data_entries = []
            buffer = ""  # To accumulate lines for a single TAF block

            i = 0
            while i < len(lines):
                line = lines[i].strip()
                
                # If the line starts with a date, treat it as a new TAF report block
                date_match = re.match(r'\d{4}/\d{2}/\d{2} \d{2}:\d{2}', line)
                if date_match:
                    if buffer:  # Parse the previous buffer before moving to the next date block
                        parsed_data = parse_taf_block(buffer)
                        if parsed_data:
                            parsed_data["Date and Time"] = date_time  # Associate with the last stored date
                            data_entries.append(parsed_data)
                        buffer = ""  # Clear buffer for the new TAF block
                    date_time = date_match.group()  # Update date_time to the new block's date

                # Continue adding lines to the buffer for the current TAF block
                if line.startswith("TAF") or buffer:
                    buffer += line + " "

                # Move to the next line
                i += 1

            # Parse the last block in the buffer after the loop ends
            if buffer:
                parsed_data = parse_taf_block(buffer)
                if parsed_data:
                    parsed_data["Date and Time"] = date_time
                    data_entries.append(parsed_data)

            # Convert the list of dictionaries to a DataFrame
            return pd.DataFrame(data_entries)
    elif data_type == "CWAM":
        with h5py.File(file_path, 'r') as file:  # Load HDF5 data
            
            return get_dataset(file)
    else:
        raise ValueError(f"Unsupported file type: {file_ext}")



In [6]:
import fetchData
import os

In [3]:
cwam_data = fetchData.load_data( 
    data_type="CWAM", 
    dataset_purpose="train", 
    path_level="part_1_220901_220924", 
    month="09", 
    day="01", 
    file_name="2022_09_01_20_00_GMT.Forecast"
)

In [7]:
print(cwam_data)
file_path = os.path.join('./', 'data', 'CWAM', f'CWAM_example.csv')
os.makedirs(os.path.dirname(file_path), exist_ok=True)
cwam_data.to_csv(file_path, index=False)
print(file_path, "saved!")

       Forecast Time (FCST) Flight Level (FLVL) Threshold (TRSH)  \
0                   FCST000             FLVL250          TRSH060   
1                   FCST000             FLVL250          TRSH060   
2                   FCST000             FLVL250          TRSH060   
3                   FCST000             FLVL250          TRSH060   
4                   FCST000             FLVL250          TRSH060   
...                     ...                 ...              ...   
104484              FCST120             FLVL450          TRSH080   
104485              FCST120             FLVL450          TRSH080   
104486              FCST120             FLVL450          TRSH080   
104487              FCST120             FLVL450          TRSH080   
104488              FCST120             FLVL450          TRSH080   

       Polygon Number (POLY)  \
0                   POLY0001   
1                   POLY0002   
2                   POLY0003   
3                   POLY0004   
4                   POL

./data/CWAM/CWAM_example.csv saved!


In [9]:
fuser_data = fetchData.load_data(
    data_type="FUSER", 
    dataset_purpose="train", 
    path_level="KATL", 
    file_name="KATL_2022-09-01.configs_data_set"
)

In [10]:
print(fuser_data)
file_path = os.path.join('./', 'data', 'FUSER', f'FUSER_example.csv')
os.makedirs(os.path.dirname(file_path), exist_ok=True)
fuser_data.to_csv(file_path, index=False)
print(file_path, "saved!")

   airport_id        data_header src_addr           datis_time  \
0        KATL  AD ATL /OS DT0052  ATLATXA  2022-09-01 00:52:00   
1        KATL  AD ATL /OS DX0152  ATLATXA  2022-09-01 01:53:00   
2        KATL  AD ATL /OS DY0152  ATLATXA  2022-09-01 02:06:00   
3        KATL  AD ATL /OS DZ0252  ATLATXA  2022-09-01 02:54:00   
4        KATL  AD ATL /OS DN0252  ATLATXA  2022-09-01 03:32:00   
5        KATL  AD ATL /OS DO0352  ATLATXA  2022-09-01 04:04:00   
6        KATL  AD ATL /OS DP0352  ATLATXA  2022-09-01 04:06:00   
7        KATL  AD ATL /OS DQ0452  ATLATXA  2022-09-01 04:54:00   
8        KATL  AD ATL /OS DR0552  ATLATXA  2022-09-01 06:06:00   
9        KATL  AD ATL /OS DS0652  ATLATXA  2022-09-01 06:53:00   
10       KATL  AD ATL /OS DT0752  ATLATXA  2022-09-01 07:56:00   
11       KATL  AD ATL /OS DU0752  ATLATXA  2022-09-01 08:15:00   
12       KATL  AD ATL /OS DV0852  ATLATXA  2022-09-01 09:08:00   
13       KATL  AD ATL /OS DW0852  ATLATXA  2022-09-01 09:38:00   
14       K

In [13]:
metar_data = fetchData.load_data(
    data_type="METAR", 
    dataset_purpose="train",  
    path_level="part_1",
    file_name="metar.20220901.00Z"
)

In [14]:
print(metar_data)
file_path = os.path.join('./', 'data', 'METAR', f'METAR_example.csv')
os.makedirs(os.path.dirname(file_path), exist_ok=True)
metar_data.to_csv(file_path, index=False)
print(file_path, "saved!")

      station datetime   cor   auto weather  temperature  dewpoint  \
0        AGGH  010000Z  None  False     SCT          NaN       NaN   
1        AGGH  010000Z  None  False     SCT          NaN       NaN   
2        AGGH  010000Z  None  False     SCT          NaN       NaN   
3        AGGH  010000Z  None  False     SCT          NaN       NaN   
4        AYMH  010000Z  None  False     HZ          20.0      17.0   
...       ...      ...   ...    ...     ...          ...       ...   
33587    ZYTX  010000Z  None  False    None         18.0       9.0   
33588    ZYTX  010000Z  None  False    None         18.0       9.0   
33589    ZYTX  010030Z  None  False    None         18.0       9.0   
33590    ZYTX  010030Z  None  False    None         18.0       9.0   
33591    ZYTX  010030Z  None  False    None         18.0       9.0   

                      date_time  \
0     2022-09-01 00:00:00+00:00   
1     2022-09-01 00:00:00+00:00   
2     2022-09-01 00:00:00+00:00   
3     2022-09-01 00

In [17]:
taf_data = fetchData.load_data( 
    data_type="TAF", 
    dataset_purpose="train",  
    file_name="taf.20220901.00Z"
)

In [18]:
print(taf_data)
file_path = os.path.join('./', 'data', 'TAF', f'TAF_example.csv')
os.makedirs(os.path.dirname(file_path), exist_ok=True)
taf_data.to_csv(file_path, index=False)
print(file_path, "saved!")

      amended  corection station issue_datetime weather  max_temp_value  \
0       False      False    ZGKL       012104Z     None            32.0   
1       False      False    ZGNN       012108Z     None            35.0   
2       False      False    ZGOW       012101Z     None            35.0   
3       False      False    ZGSZ       012120Z     None            34.0   
4       False      False    ZLXY       012107Z       BR             NaN   
...       ...        ...     ...            ...     ...             ...   
1076    False      False    TLPL       012300Z     None             NaN   
1077    False      False    TLPC       012300Z     None             NaN   
1078    False      False    KNBG           None    None             NaN   
1079    False      False    KNHK           None     SKC             NaN   
1080    False      False    KNQX           None    None             NaN   

      min_temp_value  probability  \
0               20.0          0.9   
1               26.0     